## Analyzing a csv dataset

##### This dataset was imported from the Federal Government's Open Data Portal (Brazil).
In this dataset we have data on the purchase and sale prices of ethanol and gasoline during October 2022 
- [link of portal](https://dados.gov.br/dados/conjuntos-dados/serie-historica-de-precos-de-combustiveis-e-de-glp?source=post_page-----4185005771e5--------------------------------)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 

In [2]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .appName("Spark_Dataframe")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/05/01 13:46:58 WARN Utils: Your hostname, DESKTOP-VMHJUON resolves to a loopback address: 127.0.1.1; using 192.168.92.85 instead (on interface eth0)
24/05/01 13:46:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/01 13:47:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#create a dataframe from a csv file
df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', True)
    .option('inferSchema', True)
    .option('encoding', 'ISO-8859-1')
    .csv('dataset/precos-gasolina-etanol-10.csv')
)

In [4]:
#show the schema of the dataframe
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [5]:
#create a new dataframe of prices
df_precos = (
    df
    .select('Estado - Sigla', 'Produto', 'Valor de Compra', 'Valor de Venda', 'Unidade de Medida')
)

In [6]:
#show the first 5 rows of the dataframe
df_precos.show(5)

+--------------+------------------+---------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+------------------+---------------+--------------+-----------------+
|            AL|          GASOLINA|           NULL|          6,08|       R$ / litro|
|            AL|GASOLINA ADITIVADA|           NULL|          6,08|       R$ / litro|
|            AL|            ETANOL|           NULL|          4,78|       R$ / litro|
|            AL|          GASOLINA|           NULL|          5,79|       R$ / litro|
|            AL|            ETANOL|           NULL|          4,29|       R$ / litro|
+--------------+------------------+---------------+--------------+-----------------+
only showing top 5 rows



In [7]:
# finding out if all the values ​​in the 'Valor de Compra' column are null
(
    df_precos
    .where(
        F.col('Valor de Compra').isNotNull()
    )
    .show()
 )
# the column 'Valor de Compra' was not returned because all its values ​​are null

+--------------+-------+---------------+--------------+-----------------+
|Estado - Sigla|Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+-------+---------------+--------------+-----------------+
+--------------+-------+---------------+--------------+-----------------+



In [13]:
# create a new dataframe with the 'Valor de Compra' column removed and the 'Valor de Venda' column converted to float and with the ',' replaced by '.'
df_precos = (
    df
    .select('Estado - Sigla', 'Produto',  'Valor de Venda', 'Unidade de Medida')
    .withColumn(
        'Valor de Venda',
        F.regexp_replace(F.col('Valor de Venda'), ',', '.')
        .cast('float')
    )
)